# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [61]:
# Create our features
X = df[['loan_amnt', 'int_rate', 'installment', 'annual_inc','total_acc','mths_since_recent_inq']]
# Create our target
y = df["loan_status"]

In [62]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,total_acc,mths_since_recent_inq
0,10500.0,0.1719,375.35,66000.0,14.0,1.0
1,25000.0,0.2000,929.09,105000.0,34.0,11.0
2,20000.0,0.2000,529.88,56000.0,21.0,7.0
3,10000.0,0.1640,353.55,92000.0,23.0,0.0
4,22000.0,0.1474,520.39,52000.0,28.0,0.0


In [63]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [64]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [65]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [66]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [67]:
# Scale the training and testing data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [68]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [69]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
rf_score = balanced_accuracy_score(y_test, y_pred_rf)
rf_score

0.5847776689993595

In [70]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_rf)

array([[  52,   35],
       [7329, 9789]], dtype=int64)

In [71]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.57      0.01      0.58      0.34        87
   low_risk       1.00      0.57      0.60      0.73      0.58      0.34     17118

avg / total       0.99      0.57      0.60      0.72      0.58      0.34     17205



In [73]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1924430245186124, 'annual_inc'),
 (0.17636516624787207, 'installment'),
 (0.17377815996370882, 'int_rate'),
 (0.16721812482531925, 'total_acc'),
 (0.1461364093738318, 'mths_since_recent_inq'),
 (0.14405911507065564, 'loan_amnt')]

### Easy Ensemble Classifier

In [79]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

ee = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee.fit(X_train, y_train)
y_pred_ee = ee.predict(X_test)

In [81]:
# Calculated the balanced accuracy score
score_ee = balanced_accuracy_score(y_test, y_pred_ee)
score_ee

0.5828055565627631

In [82]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ee)

array([[   50,    37],
       [ 7003, 10115]], dtype=int64)

In [83]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred_ee))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.57      0.59      0.01      0.58      0.34        87
   low_risk       1.00      0.59      0.57      0.74      0.58      0.34     17118

avg / total       0.99      0.59      0.57      0.74      0.58      0.34     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Both scores were very close but Random Forest performed slightly better

2. Which model had the best recall score?

    Easy Ensemble was slightly higher

3. Which model had the best geometric mean score?

    Equal

4. What are the top three features?

    Annual Income, Installment, Interest Rates